This is the sample script used to score on hidden dataset. We have also timed the prediction on the whole base. It will take approximately 10 min for ETA prediction of 28,000 routes.

In [6]:
import json
import requests
import pandas as pd

#conversion to pandas
def conv_(ch): 
    res = ch.strip('][').split(', ') 
    res = [s.strip('["]') for s in res]
    res_pd = pd.DataFrame(res)
    res_pd.columns = ['pred']
    res_pd['pred'] = pd.to_numeric(res_pd['pred'])
    return res_pd


#scoring function
def score_url(df): 
# URL for the web service.
    scoring_uri = 'http://41698060-7459-44a9-97d6-6b4da799ad9a.southeastasia.azurecontainer.io/score'
    # Convert to JSON string.
    input_data_json = df.to_json()
    # Set the content type.
    headers = {'Content-Type': 'application/json'}
    # Make the request and display the response.
    resp = requests.post(scoring_uri, input_data_json, headers=headers)
    # print(resp.text)
    pred = conv_(resp.text)
    return pred

#scoring function - by batch
def score_batch(df):
    pd_all = pd.DataFrame()
    if df.shape[0]>1000:
        for i in range(1,len(df)//1000+1,1):
            a = 1000*(i-1); b = i*1000
            temp = df.iloc[a:b,:]
            pred_temp = score_url(temp)
            pd_all = pd.concat([pd_all, pred_temp], axis = 0)
    else:
        pd_all= score_url(df)
        
    return pd_all

# scoring function for single entry - in dict
def score_single(input_data_json): 
# URL for the web service.
    scoring_uri = 'http://41698060-7459-44a9-97d6-6b4da799ad9a.southeastasia.azurecontainer.io/score'
    # Convert to JSON string.
    #input_data_json = df.to_json()
    # Set the content type.
    headers = {'Content-Type': 'application/json'}
    # Make the request and display the response.
    resp = requests.post(scoring_uri, input_data_json, headers=headers)
    # print(resp.text)
    pred = conv_(resp.text)
    return pred

Load the data from CSV (in pandas format)

In [2]:
inp = "D:/Kaggle/AngelHack/stupid azure/FinalModel/"
data = pd.read_csv(inp+'sample_all.csv')
sample = pd.read_csv(inp+'sample.csv')
target = pd.read_csv(inp+'target_all.csv')
single_sample = sample.iloc[0:1,:]

Final results in data frame

In [32]:
%%time
test_all = score_batch(sample)

Wall time: 1.87 s


Test with single example - just change the value after "0": for all the inputs desired. (JSON FORMAT FOR DF FORM)

In [8]:
# score_single('{"latitude_origin":{"0":1.3017751},"longitude_origin":{"0":103.7992566},"latitude_destination":{"0":1.3580012},"longitude_destination":{"0":103.8451597},"timestamp":{"0":1554809147},"hour_of_day":{"0":11},"day_of_week":{"0":1}}')

,pred
0,1146.556888


Calculation of RMSE

In [16]:
from sklearn.metrics import mean_squared_error

mean_squared_error(test_all.pred,target["eta"])**0.5

189.58138451504158